In [1]:
import redis
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.pool import NullPool
import matplotlib as plt
import matplotlib.pylab as plt
import seaborn as sns
from tqdm import tqdm
from PIL import Image
import requests
from io import BytesIO
import pickle
import zlib



In [2]:
q = '''
SELECT p.id AS pid,
       p.price AS price,
       p.flag_used AS flagUsed,
       p.category_id AS categoryId,
       pe.emergency_cnt AS emergencyCount,
       pe.comment_cnt AS commentCount,
       pe.interest AS interestCount,
       pe.pfavcnt AS favoriteCount,
       GROUP_CONCAT(bid.keyword SEPARATOR ',') as keyword_array,
       GROUP_CONCAT(bid.bid_price SEPARATOR ',') as bid_array
FROM product_info p
JOIN product_ext pe ON p.id = pe.pid
JOIN ad_set_product ap ON p.id = ap.id
JOIN ad_set_product_bid_keyword bid ON ap.ad_set_id = bid.ad_set_product_id
group by 1, 2, 3, 4, 5, 6, 7, 8
'''
t = pd.read_sql(q, con=mysql)

In [7]:
type(t['keyword_array'][0])

str

In [ ]:
class ProductFeature(object):
    def __init__(self, pid, keyword, bidPrice):
        self._pid = pid
        self._keyword = keyword
        self._bidPrice
        

In [2]:
r = redis.Redis(host='127.0.0.1', port=6379, db=1, decode_responses=True)
# r = redis.StrictRedis(host="localhost", port=6379, db=0)

In [ ]:
DROP TABLE IF EXISTS workspace.emily_product_temp;

CREATE TABLE workspace.emily_product_temp AS
SELECT i.content_id,
	   p.uid 
FROM bun_log_db.app_event_type_impression i
JOIN service1_quicket.product_info p ON i.content_id = p.id
WHERE i.YEAR||i.MONTH||i.DAY||i.HOUR = '2020110110'
  AND i.ref_source IN ('ad_super_up',
                     'ad_super_up_shop',
                     'power_up')
  AND i.ref_term = '몽클레어 패딩';

In [3]:
q = '''
SELECT p.id AS pid,
       p.uid AS uid,
       p.price,
       p.flag_used,
       p.category_id,
       left(p.category_id, 3) AS sub_category_id,
       left(p.category_id, 6) AS second_sub_category_id,
       pe.emergency_cnt,
       pe.comment_cnt,
       pe.interest,
       pe.pfavcnt
FROM service1_quicket.product_info p
JOIN service1_quicket.product_ext pe ON p.id = pe.pid
JOIN workspace.emily_product_temp t ON t.content_id = p.id
'''
product = pd.read_sql(q, con=bun_dw)

In [4]:
str_col = ['flag_used', 'category_id', 'sub_category_id', 'second_sub_category_id']
for col in str_col:
    product[col] = product[col].astype(str)

In [5]:
q = '''
SELECT u.id AS uid,
       CASE ui.gender
           WHEN '1' THEN '1'
           WHEN '2' THEN '2'
           WHEN '0' THEN '2'
           ELSE '0'
       END AS gender,
       coalesce(NULL, (to_char(getdate(), 'yyyy') - ui.birth_year), '0') AS age,
       u.favorite_count,
       u.grade,
       u.item_count,
       u.interest,
       u.review_count,
       u.comment_count,
       ue.follower_cnt,
       ue.following_cnt,
       ue.bunpay_count,
       ue.parcel_post_count,
       ue.transfer_count,
       ue.bunp_meet_count
FROM service1_quicket.user_ u
LEFT JOIN service1_quicket.user_identification_v2 ui ON u.id = ui.uid
JOIN service1_quicket.user_extra_info ue ON u.id = ue.uid
WHERE u.id IN
    (
     SELECT p.uid::varchar
     FROM workspace.emily_product_temp p
     UNION 
     SELECT i.user_id AS uid
     FROM bun_log_db.app_event_type_impression i
     JOIN workspace.emily_product_temp p ON i.content_id = p.content_id
     WHERE YEAR||MONTH||DAY||HOUR = '2020110110'
    );
'''
user = pd.read_sql(q, con=bun_dw)

In [6]:
str_col = ['gender', 'age']
for col in str_col:
    user[col] = user[col].astype(str)

In [7]:
product['uid'] = product['uid'].astype(str)
user['uid'] = user['uid'].astype(str)
owners = product['uid'].unique()
uids = user['uid'].unique()

for i in owners:
    if i not in uids:
        print(i)

In [217]:
product = product.drop_duplicates()
user = user.drop_duplicates()


In [265]:
product

,pid,uid,price,flag_used,category_id,sub_category_id,second_sub_category_id,emergency_cnt,comment_cnt,interest,pfavcnt
0,101338381,209181,35000,2,320030100,320,320030,3,10,20073,711
5,112081029,4873981,48000,2,310060010,310,310060,0,0,258,16
6,112200270,10293857,28000,2,320150200,320,320150,0,4,20879,485
14,112246996,10293857,57000,2,320080100,320,320080,0,1,29700,790
19,109228644,4959074,35000,2,320050100,320,320050,0,4,6685,151
...,...,...,...,...,...,...,...,...,...,...,...
2035,132830002,5082078,105000,2,320030200,320,320030,0,0,782,30
2036,93697510,644784,29000,2,320030100,320,320030,1,28,6122,364
2039,112248497,10293857,31000,2,320150300,320,320150,0,0,4572,131
2046,134011494,7147284,250000,13,320040500,320,320040,0,0,172,4


In [8]:
product = product.drop_duplicates()

products = []
for i, j in product.iterrows():
    d = dict()
    d['productId'] = j['pid']
    d['productOwner'] = int(j['uid'])
    products.append(d)

In [9]:
len(products)

909

In [10]:
user

,uid,gender,age,favorite_count,grade,item_count,interest,review_count,comment_count,follower_cnt,following_cnt,bunpay_count,parcel_post_count,transfer_count,bunp_meet_count
0,6119967,2,43.0,165,1883,353,6268,192,77,0,4,21,559,10,10
1,4161033,1,31.0,68,11,266,1709,2,2,0,5,1,0,0,1
2,1744269,1,28.0,2,13,48,3702,3,5,0,4,1,0,0,0
3,6339297,0,0.0,0,0,2,133,0,2,0,1,0,0,0,0
4,2979004,0,0.0,0,0,2,122,0,2,0,17,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7200,1874346,0,0.0,0,10,0,0,1,3,0,0,0,0,0,0
7201,12919197,2,24.0,0,0,1,0,0,0,0,0,0,0,0,0
7202,3936935,0,0.0,2,0,5,417,0,1,0,45,0,0,0,0
7203,386940,0,0.0,1,10,10,1167,1,3,0,0,0,0,0,0


In [11]:
json = {
    "uid": 6818710,
    "products": products
}


In [373]:
print(json)

{'uid': 6818710, 'products': [{'productId': 111347557, 'productOwner': 3526851}, {'productId': 110469621, 'productOwner': 5096602}, {'productId': 122180258, 'productOwner': 3526851}, {'productId': 123368259, 'productOwner': 3526851}, {'productId': 111164428, 'productOwner': 4552060}, {'productId': 114679557, 'productOwner': 679093}, {'productId': 131278357, 'productOwner': 3326323}, {'productId': 133723978, 'productOwner': 10473216}, {'productId': 133443143, 'productOwner': 12272943}, {'productId': 133936466, 'productOwner': 10293857}, {'productId': 113484087, 'productOwner': 679093}, {'productId': 133566572, 'productOwner': 12057139}, {'productId': 134304271, 'productOwner': 149955}, {'productId': 134346276, 'productOwner': 12136429}, {'productId': 135228635, 'productOwner': 7482151}, {'productId': 134412090, 'productOwner': 12136429}, {'productId': 115316845, 'productOwner': 3526851}, {'productId': 135222516, 'productOwner': 12136429}, {'productId': 136126160, 'productOwner': 9248815

In [12]:
from tqdm import tqdm
import pickle

uids = user['uid']
for uid in tqdm(uids):
    dictionary = user[user['uid'] == uid].drop('uid', axis=1).to_dict('recoreds')[0]
#     temp = pickle.dumps(dictionary)
    r.set('uid:'+str(uid), dictionary)
#     r.set("uid:" + str(uid), zlib.compress(pickle.dumps(user[user['uid']==uid].drop('uid', axis=1))))
#     r.set("uid:" + str(uid), user[user['uid'] == uid].drop('uid', axis=1).to_msgpack(compress='zlib'))

  0%|          | 0/7205 [00:00<?, ?it/s]/Users/emilyPark/.pyenv/versions/bun-analytics/lib/python3.7/site-packages/pandas/core/frame.py:1490: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  FutureWarning,
100%|██████████| 7205/7205 [00:13<00:00, 545.31it/s]


In [376]:
pids = product['pid']
for pid in tqdm(pids):
    dictionary = product[product['pid'] == pid].drop(['pid', 'uid'], axis=1).to_dict('records')[0]
    r.set('pid:'+str(pid), dictionary)
    
#     r.set("pid:" + str(pid), zlib.compress(pickle.dumps(product[product['pid']==pid].drop(['pid', 'uid'], axis=1))))
#     r.set("uid:" + str(uid), user[user['uid'] == uid].drop('uid', axis=1).to_msgpack(compress='zlib'))

100%|██████████| 909/909 [00:01<00:00, 710.02it/s]


In [16]:
import json

with open('result_json.txt', 'r', encoding='utf-8') as f:
    result = json.loads(f.read())

In [21]:
result_dict = {'pclick': [], 'productId': []}
result_data = result['data']
for i in result_data:
    result_dict['pclick'] = result_dict['pclick'] + [i['pclick']]
    result_dict['productId'] = result_dict['productId'] + [i['productId']]
result16 = pd.DataFrame.from_dict(result_dict)

In [32]:
import json

with open('result_31.txt', 'r', encoding='utf-8') as f:
    result31 = json.loads(f.read())

result_dict = {'pclick': [], 'productId': []}
result_data = result31['data']
for i in result_data:
    result_dict['pclick'] = result_dict['pclick'] + [i['pclick']]
    result_dict['productId'] = result_dict['productId'] + [i['productId']]
    
result_31 = pd.DataFrame.from_dict(result_dict)

In [34]:
import json

with open('result_43.txt', 'r', encoding='utf-8') as f:
    result31 = json.loads(f.read())

result_dict = {'pclick': [], 'productId': []}
result_data = result31['data']
for i in result_data:
    result_dict['pclick'] = result_dict['pclick'] + [i['pclick']]
    result_dict['productId'] = result_dict['productId'] + [i['productId']]
    
result_43 = pd.DataFrame.from_dict(result_dict)

In [31]:
result16.describe()

,pclick,productId
count,909.000000,9.090000e+02
mean,0.022805,1.312785e+08
std,0.024529,1.126472e+07
min,0.001233,3.752043e+07
25%,0.007603,1.332788e+08
50%,0.015756,1.354939e+08
75%,0.027417,1.363264e+08
max,0.230406,1.368582e+08


In [37]:
result16[result16['pclick'] == result16['pclick'].max()]

,pclick,productId
218,0.230406,136820961


In [33]:
result_31.describe()

,pclick,productId
count,909.000000,9.090000e+02
mean,0.028335,1.312785e+08
std,0.026724,1.126472e+07
min,0.002065,3.752043e+07
25%,0.010920,1.332788e+08
50%,0.021388,1.354939e+08
75%,0.037222,1.363264e+08
max,0.301501,1.368582e+08


In [38]:
result_31[result_31['pclick'] == result_31['pclick'].max()]

,pclick,productId
364,0.301501,112247051


In [35]:
result_43.describe()

,pclick,productId
count,909.000000,9.090000e+02
mean,0.010164,1.312785e+08
std,0.008845,1.126472e+07
min,0.000606,3.752043e+07
25%,0.004471,1.332788e+08
50%,0.007408,1.354939e+08
75%,0.013135,1.363264e+08
max,0.060571,1.368582e+08


In [39]:
result_43[result_43['pclick'] == result_43['pclick'].max()]

,pclick,productId
290,0.060571,136199547


In [44]:
re = redis.Redis(host='127.0.0.1', port=6379, db=0, decode_responses=True)

In [48]:
d = {'feature': 'refterm', 'feature2': 'user'}
len(d)

2

In [3]:
# -*- coding: utf-8 -*-
import boto3
import json
kinesis_client = boto3.client('kinesis', region_name='ap-northeast-2')
stream_name = 'bun-ad-prediction-test'

def put_to_stream(thing_id, payload):
    put_response = kinesis_client.put_record(
        StreamName=stream_name,
        Data=json.dumps(payload),
        PartitionKey=thing_id)
    print(put_response)

    
thing_id = 'foo'
payload = {
   "data":{
      "adSet":{
         "id":14,
         "uid":11423083,
         "name":"광고 세트 117",
         "status":"IN_PROGRESS",
         "targetGender":"WOMAN",
         "spentType":"FAST_PUBLISHED",
         "adType":"SA",
         "payType":"CPC",
         "totalBudget":18000,
         "paidBudget":0,
         "freeBudget":18000,
         "totalPoint":0,
         "paidPoint":0,
         "freePoint":0,
         "creatorUid":9145992,
         "modifierUid":9145992,
         "dailyBudget":[
            {
               "startDateTime":"2020-12-22T20:00:00",
               "endDateTime":"2020-12-22T23:59:59",
               "paidBudget":0,
               "freeBudget":391,
               "paidPoint":0,
               "freePoint":0
            },
            {
               "startDateTime":"2020-12-01T20:00:00",
               "endDateTime":"2020-12-01T23:59:59",
               "paidBudget":0,
               "freeBudget":348,
               "paidPoint":0,
               "freePoint":0
            }
         ],
         "startDateTime":"2020-12-22T20:00:00",
         "endDateTime":"2020-12-30T11:59:59",
         "targetAddressList":[
            2,
            1
         ],
         "targetAges":[
            {
               "minAge":11,
               "maxAge":20
            },
            {
               "minAge":21,
               "maxAge":30
            }
         ],
         "adSetProducts":[
            {
               "adSetProductId":1,
               "pid":138189233,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
             {
               "adSetProductId":2,
               "pid":139270343,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
             {
               "adSetProductId":3,
               "pid":139273861,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":4,
               "pid":139292489,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":5,
               "pid":139292588,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":6,
               "pid":139292745,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":7,
               "pid":139292873,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":8,
               "pid":139293186,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":9,
               "pid":139293559,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":10,
               "pid":139294020,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            },
            {
               "adSetProductId":11,
               "pid":139447202,
               "status":"IN_PROGRESS",
               "keywords":[
                  {
                     "id":91732,
                     "adSetProductId":22035,
                     "keyword":"아이폰12",
                     "bidPrice":605,
                     "active":True
                  }
               ],
               "name":"나닷"
            }
         ]
      },
      "before":{
         "id":309,
         "adSetProducts":[
            {
               "adSetProductId":1178,
               "pid":2000001538,
               "keywords":[
                  {
                     "id":4443
                  },
                  {
                     "id":4291,
                     "adSetProductId":1178,
                     "keyword":"류류류34034",
                     "active":True
                  }
               ],
               "name":"테스"
            }
         ]
      }
   },
   "eventType":"ADSET_MODIFIED",
   "eventTime":"2020-12-01T18:44:37.755",
   "actorUid":9145992
}
put_to_stream(thing_id, payload)

{'ShardId': 'shardId-000000000000', 'SequenceNumber': '49612982131895676197735180863658468424866432904591310850', 'ResponseMetadata': {'RequestId': 'db24c515-f4f6-0ccc-8378-54a022ebe196', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'db24c515-f4f6-0ccc-8378-54a022ebe196', 'x-amz-id-2': 'NREXIa9d6mrhjF9mKVGENTBal8kxU2dr/gC0ZS4nX8ts4HVZhL+99dA4QGjBoFeorH3ayD1GPfYSkCaKwkMBO3hPNdw11kBS', 'date': 'Wed, 23 Dec 2020 00:29:41 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '110'}, 'RetryAttempts': 0}}
